In [4]:
#pip install pyjanitor

In [5]:
import pandas as pd
import numpy as np
import seaborn as sns
import datetime as dt
import matplotlib.pyplot as plt
from matplotlib.dates import DateFormatter
import matplotlib.dates as mdates
from scipy.stats import shapiro
from numpy.random import randn
import scipy.stats as stats
from __future__ import division
import scipy.stats
from datetime import datetime, timedelta
#import plotly.figure_factory as ff
import matplotlib, random
import re
import janitor

### Loading the data

In [4]:
df=pd.read_csv("station_db.csv")
dregion=pd.read_csv("coord_region.csv")

/tmp/ipykernel_460/796669064.py:1: DtypeWarning: Columns (6,7,8,9,10,11,12,13) have mixed types. Specify dtype option on import or set low_memory=False.
  df=pd.read_csv("station_db.csv")


In [5]:
## remove negative value in precipitation
j=df["prec"]<0
df["prec"][j]=None

/tmp/ipykernel_460/3016797823.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["prec"][j]=None


In [6]:
## Remove special character in the data
df["srad"] = df["srad"].replace("-", None, regex=True)
df["tmax"] = df["tmax"].replace("-", None, regex=True)
df["tmax"] = df["tmax"].replace(True, None, regex=True)
df["tmax"] = df["tmax"].replace("TRUE", None, regex=True)
df["tmax"] = df["tmax"].replace('   ',"", regex=True)
df["tmin"] = df["tmin"].replace("TRUE", None, regex=True)
df["tmin"] = df["tmin"].replace("-", None, regex=True)
df["wind"]= df["wind"].replace("-", None, regex=True)
## data type 
df["srad"]=df["srad"].astype(float)
df["tmax"]=df["tmax"].astype(float)
df["tmin"]=df["tmin"].astype(float)
df["wind"]=df["wind"].astype(float)

In [7]:
# add Region and country in the station data
dregion=dregion[["name","country","Region"]]
df=df.merge(dregion,how='left', left_on='name', right_on='name')

In [4]:
df=pd.read_csv("df_outlier_remove.csv")

/tmp/ipykernel_446/3653840344.py:1: DtypeWarning: Columns (7,9,11,13,14,15) have mixed types. Specify dtype option on import or set low_memory=False.
  df=pd.read_csv("df_outlier_remove.csv")


In [5]:
#dG=df[(df["N_Outlier "]==0) & (df["Region"]=="South Region")]

In [6]:
#len(dG["name"].unique())

In [7]:
def gaps(var,data):
    L,L1,L2,L3,L4,S=[],[],[],[],[],[]
    data['date'] = pd.to_datetime(data['date'])
    station_name=data["name"].unique()
    for station in station_name :
        df=data[(data["name"]==station) & (data[var].notnull())]
        df=df.sort_values(by="date")
        df=df[df["date"].notnull()]
        dates = pd.DataFrame(pd.date_range(start=df["date"].min(), end=df["date"].max()), columns=['date'])
        d1=dates.merge(df,how='left', left_on='date', right_on='date')
        M=len(d1[d1[var].isnull()])
        d1=d1[d1[var].notnull()]
        start,end =df["date"].min(),df["date"].max()
        dgap= d1['date'].diff()[1:]
        gaps = dgap[dgap > timedelta(days=1)]
        C=df["country"].unique()
        L.append(len(gaps))
        L1.append(start)
        L2.append(end)
        L3.append(station)
        L4.append(M)
        S.append(C)
    d=pd.DataFrame(L3,columns=["name"])
    d["start_date"]=L1
    d["End_date"]=L2
    d["Number_gap"]=L
    d["Number_Missing"]=L4
    d["country"]=S
    #d["class"]=LL
    
    return d

In [8]:
## Add class for each station
dd=gaps("prec",df)
dd["Class"]= np.where(dd["Number_gap"]==0,"A",
                     np.where(((0<dd["Number_Missing"]) & (dd["Number_Missing"]<=10)) ,"B","C"))

In [10]:
#dd.to_csv("prec_Gap_evaluation" + ".csv",index=False)

In [11]:
dd["Class"].value_counts()

Class
C    98
A    62
B    16
Name: count, dtype: int64

In [12]:
dG=dd[dd["Class"]=="A"]
dG=dG.sort_values(by=["start_date","End_date"]).reset_index(drop=True)

In [13]:
dG.tail(50)

,name,start_date,End_date,Number_gap,Number_Missing,country,Class
12,TinderetTea,1950-01-01,2004-12-31,0,0,[Kenya],A
13,Rusinga,1950-01-01,2004-12-31,0,0,[Kenya],A
14,Muhuru,1950-01-01,2004-12-31,0,0,[Tanzania],A
15,KisumuAirport,1950-01-01,2008-12-31,0,0,[Kenya],A
16,Kericho,1950-01-01,2008-12-31,0,0,[Kenya],A
17,Eldoret,1950-01-01,2008-12-31,0,0,[Kenya],A
18,KitaleMet,1950-01-01,2009-12-31,0,0,[Kenya],A
19,Alupe,1950-01-02,2004-12-31,0,0,[Kenya],A
20,Bulawayo_goetz observatory,1951-07-01,2001-12-31,0,0,[Zimbabwe],A
21,Masvingo,1951-07-01,2001-12-31,0,0,[nan],A


In [14]:
# Add class in the data
df_prec=df[["name","date","prec","Region","country"]]
df_prec=df_prec.merge(dd[["name","Class"]],how='left', left_on='name', right_on='name')

In [15]:
df_prec['date'] = pd.to_datetime(df_prec['date'])
df_prec=df_prec.sort_values(by="date")
def extract(station,var):
    d=df_prec[(df_prec["name"]==station) & (df_prec[var].notnull())]
    dd=d.sort_values(by="date")
    dd=dd[["date",var]]
    dd.rename(columns = {var:station},inplace = True)
    return dd

In [17]:
# filter continu stations base on the start and end date
df_prec1=df_prec[((df_prec["date"]>="1998-01-01")& (df_prec["date"]<="2018-12-31"))  ]
#df_prec1=df_prec[(df_prec["Class"]=="A") ]

In [18]:
dates = pd.DataFrame(pd.date_range(start=df_prec1["date"].min(), end=df_prec1["date"].max()), columns=['date'])
station=df_prec1["name"].unique()
d=dates["date"]
for S in station :
    d1=extract(S,"prec")
    d1.drop_duplicates(subset="date",keep="first", inplace=True)
    d2=dates.merge(d1,how='left', left_on='date', right_on='date')
    d2=d2.sort_values(by="date")
    d2=d2[[S]]
    dd1=pd.concat([d,d2],axis=1)
    d=dd1
max_number_of_nas = 0
dNNAS = dd1.loc[:, (dd1.isnull().sum(axis=0) <= max_number_of_nas)]    
#dNNAS.to_csv("prec_1981_2010" + ".csv",index=False)

In [38]:
#d.to_csv("prec_classC" + ".csv",index=False)

In [37]:
dNNAS.columns

Index(['date', 'Bruyns Hill'], dtype='object')

In [23]:
#dNNAS.corr()

In [5]:
# Test for seasonality
#from pandas.plotting import autocorrelation_plot

# Draw Plot
#plt.rcParams.update({'figure.figsize':(10,6), 'figure.dpi':120})
#autocorrelation_plot(dNNAS['Takoradi'].tolist())

In [4]:
#pd.read_csv("coord_region.csv")

In [3]:
#pp=pd.read_csv("Qualitative_Tanzania.csv")
#pp.to_excel("Qulit_Tanzania.xlsx",index=False)